## Tensorflow使用TensorBoard可视化查看指标

使用TensorBoard可以打开一个网页：
1. 查看指标随着batch的训练不断变化
2. 查看网络结构图

In [1]:
import tensorflow as tf
from tensorflow import keras

# 使用sklearn现成的波斯顿房价数据集
from sklearn.datasets import fetch_california_housing

# 做训练集测试集验证集拆分
from sklearn.model_selection import train_test_split

# 对数值数据归一化
from sklearn.preprocessing import StandardScaler

### 1. 下载sklearn的波士顿房价数据集

In [2]:
# 使用这个函数下载波斯顿房价数据集
housing = fetch_california_housing()

In [3]:
# 分割训练集、测试集、验证集
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [4]:
# 使用sklearn对数值数据做归一化
scaler = StandardScaler()
# 在训练集上做fit和transform
X_train = scaler.fit_transform(X_train)

# 在测试集和验证集上做transform
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### 2. 把数据分割为Wide部分输入和Deep部分输入

In [5]:
# A是0~4共5列，B是2~7共6列
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]

# valid和test做同样的处理
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

### 3. 搭建WideDeep模型

In [6]:
# 输入：wide和deep
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")

# 构造deep部分
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
hidden3 = keras.layers.Dense(30, activation="relu")(hidden2)

# 合并wide和deep
concat = keras.layers.concatenate([input_A, hidden3])

# 输出层
output = keras.layers.Dense(1, name="output")(concat)

# 构造model，注意多输入，单输出
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

### 4. 模型编译、训练

In [7]:
# 模型编译，指定多个指标
model.compile(
    loss="mse", 
    optimizer=keras.optimizers.SGD(lr=1e-3),
    metrics=[tf.keras.metrics.MeanSquaredError(),
             tf.keras.metrics.RootMeanSquaredError(),
             tf.keras.metrics.MeanAbsoluteError()]
)

In [8]:
# TensorBoard
!rm -rf ./logs/*
tensorboard_cb = tf.keras.callbacks.TensorBoard("./logs")

In [ ]:
# 启动方法：tensorboard --logdir=./logs --port=9998 --bind_all

In [ ]:
# 模型训练
model.fit(
    (X_train_A, X_train_B), 
    y_train, 
    epochs=1000,
    validation_data=((X_valid_A, X_valid_B), y_valid),
    callbacks=[tensorboard_cb]
)

Epoch 1/1000
363/363 [==============================] - 1s 2ms/step - loss: 1.6456 - mean_squared_error: 1.6456 - root_mean_squared_error: 1.2828 - mean_absolute_error: 0.8964 - val_loss: 0.8725 - val_mean_squared_error: 0.8725 - val_root_mean_squared_error: 0.9341 - val_mean_absolute_error: 0.6844
Epoch 2/1000
363/363 [==============================] - 1s 2ms/step - loss: 0.8109 - mean_squared_error: 0.8109 - root_mean_squared_error: 0.9005 - mean_absolute_error: 0.6621 - val_loss: 0.7463 - val_mean_squared_error: 0.7463 - val_root_mean_squared_error: 0.8639 - val_mean_absolute_error: 0.6398
Epoch 3/1000
363/363 [==============================] - 1s 3ms/step - loss: 0.7250 - mean_squared_error: 0.7250 - root_mean_squared_error: 0.8515 - mean_absolute_error: 0.6291 - val_loss: 0.6903 - val_mean_squared_error: 0.6903 - val_root_mean_squared_error: 0.8308 - val_mean_absolute_error: 0.6085
Epoch 4/1000
363/363 [==============================] - 2s 4ms/step - loss: 0.6796 - mean_squared_er

363/363 [==============================] - 1s 2ms/step - loss: 0.4725 - mean_squared_error: 0.4725 - root_mean_squared_error: 0.6874 - mean_absolute_error: 0.4997 - val_loss: 0.4777 - val_mean_squared_error: 0.4777 - val_root_mean_squared_error: 0.6912 - val_mean_absolute_error: 0.4929
Epoch 29/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.4691 - mean_squared_error: 0.4691 - root_mean_squared_error: 0.6849 - mean_absolute_error: 0.4983 - val_loss: 0.4745 - val_mean_squared_error: 0.4745 - val_root_mean_squared_error: 0.6888 - val_mean_absolute_error: 0.4961
Epoch 30/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.4652 - mean_squared_error: 0.4652 - root_mean_squared_error: 0.6821 - mean_absolute_error: 0.4957 - val_loss: 0.4711 - val_mean_squared_error: 0.4711 - val_root_mean_squared_error: 0.6863 - val_mean_absolute_error: 0.4944
Epoch 31/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.4624 - mean_squared_error: 0.462

363/363 [==============================] - 0s 1ms/step - loss: 0.4116 - mean_squared_error: 0.4116 - root_mean_squared_error: 0.6415 - mean_absolute_error: 0.4619 - val_loss: 0.4193 - val_mean_squared_error: 0.4193 - val_root_mean_squared_error: 0.6476 - val_mean_absolute_error: 0.4622
Epoch 56/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.4102 - mean_squared_error: 0.4102 - root_mean_squared_error: 0.6405 - mean_absolute_error: 0.4608 - val_loss: 0.4211 - val_mean_squared_error: 0.4211 - val_root_mean_squared_error: 0.6489 - val_mean_absolute_error: 0.4707
Epoch 57/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.4083 - mean_squared_error: 0.4083 - root_mean_squared_error: 0.6390 - mean_absolute_error: 0.4603 - val_loss: 0.4162 - val_mean_squared_error: 0.4162 - val_root_mean_squared_error: 0.6451 - val_mean_absolute_error: 0.4573
Epoch 58/1000
363/363 [==============================] - 1s 2ms/step - loss: 0.4069 - mean_squared_error: 0.406

363/363 [==============================] - 0s 1ms/step - loss: 0.3758 - mean_squared_error: 0.3758 - root_mean_squared_error: 0.6130 - mean_absolute_error: 0.4376 - val_loss: 0.3832 - val_mean_squared_error: 0.3832 - val_root_mean_squared_error: 0.6190 - val_mean_absolute_error: 0.4405
Epoch 83/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3747 - mean_squared_error: 0.3747 - root_mean_squared_error: 0.6121 - mean_absolute_error: 0.4375 - val_loss: 0.3831 - val_mean_squared_error: 0.3831 - val_root_mean_squared_error: 0.6190 - val_mean_absolute_error: 0.4381
Epoch 84/1000
363/363 [==============================] - 1s 4ms/step - loss: 0.3739 - mean_squared_error: 0.3739 - root_mean_squared_error: 0.6115 - mean_absolute_error: 0.4368 - val_loss: 0.3808 - val_mean_squared_error: 0.3808 - val_root_mean_squared_error: 0.6171 - val_mean_absolute_error: 0.4324
Epoch 85/1000
363/363 [==============================] - 1s 2ms/step - loss: 0.3727 - mean_squared_error: 0.372

363/363 [==============================] - 0s 1ms/step - loss: 0.3523 - mean_squared_error: 0.3523 - root_mean_squared_error: 0.5936 - mean_absolute_error: 0.4213 - val_loss: 0.3581 - val_mean_squared_error: 0.3581 - val_root_mean_squared_error: 0.5984 - val_mean_absolute_error: 0.4173
Epoch 110/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3520 - mean_squared_error: 0.3520 - root_mean_squared_error: 0.5933 - mean_absolute_error: 0.4209 - val_loss: 0.3578 - val_mean_squared_error: 0.3578 - val_root_mean_squared_error: 0.5981 - val_mean_absolute_error: 0.4164
Epoch 111/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3517 - mean_squared_error: 0.3517 - root_mean_squared_error: 0.5931 - mean_absolute_error: 0.4207 - val_loss: 0.3566 - val_mean_squared_error: 0.3566 - val_root_mean_squared_error: 0.5971 - val_mean_absolute_error: 0.4199
Epoch 112/1000
363/363 [==============================] - 1s 2ms/step - loss: 0.3506 - mean_squared_error: 0.

...

Epoch 298/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3084 - mean_squared_error: 0.3084 - root_mean_squared_error: 0.5553 - mean_absolute_error: 0.3867 - val_loss: 0.3255 - val_mean_squared_error: 0.3255 - val_root_mean_squared_error: 0.5706 - val_mean_absolute_error: 0.3957
Epoch 299/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3082 - mean_squared_error: 0.3082 - root_mean_squared_error: 0.5551 - mean_absolute_error: 0.3867 - val_loss: 0.3220 - val_mean_squared_error: 0.3220 - val_root_mean_squared_error: 0.5675 - val_mean_absolute_error: 0.3899
Epoch 300/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3078 - mean_squared_error: 0.3078 - root_mean_squared_error: 0.5548 - mean_absolute_error: 0.3865 - val_loss: 0.3222 - val_mean_squared_error: 0.3222 - val_root_mean_squared_error: 0.5676 - val_mean_absolute_error: 0.3849
Epoch 301/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3077 - mean_sq

363/363 [==============================] - 0s 1ms/step - loss: 0.3052 - mean_squared_error: 0.3052 - root_mean_squared_error: 0.5524 - mean_absolute_error: 0.3850 - val_loss: 0.3203 - val_mean_squared_error: 0.3203 - val_root_mean_squared_error: 0.5660 - val_mean_absolute_error: 0.3886
Epoch 326/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3058 - mean_squared_error: 0.3058 - root_mean_squared_error: 0.5530 - mean_absolute_error: 0.3845 - val_loss: 0.3197 - val_mean_squared_error: 0.3197 - val_root_mean_squared_error: 0.5654 - val_mean_absolute_error: 0.3869
Epoch 327/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3060 - mean_squared_error: 0.3060 - root_mean_squared_error: 0.5531 - mean_absolute_error: 0.3845 - val_loss: 0.3189 - val_mean_squared_error: 0.3189 - val_root_mean_squared_error: 0.5647 - val_mean_absolute_error: 0.3847
Epoch 328/1000
363/363 [==============================] - 1s 1ms/step - loss: 0.3063 - mean_squared_error: 0.

363/363 [==============================] - 0s 1ms/step - loss: 0.3021 - mean_squared_error: 0.3021 - root_mean_squared_error: 0.5497 - mean_absolute_error: 0.3822 - val_loss: 0.3200 - val_mean_squared_error: 0.3200 - val_root_mean_squared_error: 0.5657 - val_mean_absolute_error: 0.3905
Epoch 353/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3034 - mean_squared_error: 0.3034 - root_mean_squared_error: 0.5508 - mean_absolute_error: 0.3828 - val_loss: 0.3197 - val_mean_squared_error: 0.3197 - val_root_mean_squared_error: 0.5654 - val_mean_absolute_error: 0.3909
Epoch 354/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3029 - mean_squared_error: 0.3029 - root_mean_squared_error: 0.5503 - mean_absolute_error: 0.3822 - val_loss: 0.3182 - val_mean_squared_error: 0.3182 - val_root_mean_squared_error: 0.5641 - val_mean_absolute_error: 0.3862
Epoch 355/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3032 - mean_squared_error: 0.

363/363 [==============================] - 0s 1ms/step - loss: 0.3001 - mean_squared_error: 0.3001 - root_mean_squared_error: 0.5478 - mean_absolute_error: 0.3797 - val_loss: 0.3163 - val_mean_squared_error: 0.3163 - val_root_mean_squared_error: 0.5624 - val_mean_absolute_error: 0.3823
Epoch 380/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3016 - mean_squared_error: 0.3016 - root_mean_squared_error: 0.5492 - mean_absolute_error: 0.3805 - val_loss: 0.3223 - val_mean_squared_error: 0.3223 - val_root_mean_squared_error: 0.5677 - val_mean_absolute_error: 0.3956
Epoch 381/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.3009 - mean_squared_error: 0.3009 - root_mean_squared_error: 0.5485 - mean_absolute_error: 0.3805 - val_loss: 0.3233 - val_mean_squared_error: 0.3233 - val_root_mean_squared_error: 0.5686 - val_mean_absolute_error: 0.3942
Epoch 382/1000
363/363 [==============================] - 2s 5ms/step - loss: 0.3006 - mean_squared_error: 0.

363/363 [==============================] - 0s 1ms/step - loss: 0.2996 - mean_squared_error: 0.2996 - root_mean_squared_error: 0.5474 - mean_absolute_error: 0.3792 - val_loss: 0.3165 - val_mean_squared_error: 0.3165 - val_root_mean_squared_error: 0.5626 - val_mean_absolute_error: 0.3881
Epoch 407/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.2989 - mean_squared_error: 0.2989 - root_mean_squared_error: 0.5468 - mean_absolute_error: 0.3789 - val_loss: 0.3139 - val_mean_squared_error: 0.3139 - val_root_mean_squared_error: 0.5603 - val_mean_absolute_error: 0.3806
Epoch 408/1000
363/363 [==============================] - 0s 1ms/step - loss: 0.2988 - mean_squared_error: 0.2988 - root_mean_squared_error: 0.5466 - mean_absolute_error: 0.3783 - val_loss: 0.3155 - val_mean_squared_error: 0.3155 - val_root_mean_squared_error: 0.5617 - val_mean_absolute_error: 0.3867
Epoch 409/1000
363/363 [==============================] - 1s 1ms/step - loss: 0.2976 - mean_squared_error: 0.